# AISingersCoverGen Colab

Полный рабочий ноутбук для запуска веб‑интерфейса **AISingersCoverGen** в Google Colab. Он клонирует репозиторий, ставит зависимости (без ошибок mega.py), скачивает базовые модели и запускает Gradio.


## Как пользоваться
1. Выполните ячейку с установкой зависимостей — она клонирует проект и ставит все библиотеки.
2. Запустите скачивание моделей (hubert, rmvpe и MDXNET).
3. При желании скачайте свою RVC‑модель по ссылке (Mega и обычные ссылки поддерживаются).
4. Запустите веб‑интерфейс — получите публичную ссылку вида `https://xxxx.gradio.live`.


In [ ]:
# @title 1. Клонирование репозитория и установка зависимостей
# Если хотите использовать свой форк, замените URL ниже.
REPO_URL = "https://github.com/varaslaw/AiCoverSing2.git"  # @param {type:"string"}
REPO_BRANCH = "work"  # @param {type:"string"}

import os, subprocess, sys
from pathlib import Path

if not Path("AiCoverSing2").exists():
    subprocess.run([
        "git", "clone", "--depth", "1", "-b", REPO_BRANCH, REPO_URL, "AiCoverSing2"
    ], check=True)

os.chdir("AiCoverSing2")

# Обновляем pip и ставим CUDA-сборку PyTorch (Colab использует CUDA 12.1)
subprocess.run([sys.executable, "-m", "pip", "install", "--upgrade", "pip"], check=True)
subprocess.run([
    sys.executable, "-m", "pip", "install", "torch==2.3.1", "torchaudio==2.3.1",
    "--index-url", "https://download.pytorch.org/whl/cu121"
], check=True)

# Остальные зависимости из requirements.txt (mega.py включён)
subprocess.run([
    sys.executable, "-m", "pip", "install", "--no-cache-dir", "-r", "requirements.txt"
], check=True)

# ffmpeg нужен для обработки аудио
subprocess.run(["apt-get", "update"], check=True)
subprocess.run(["apt-get", "install", "-y", "ffmpeg"], check=True)

print("
Установка завершена. Текущая директория:", Path.cwd())


In [ ]:
# @title 2. Скачать базовые модели (hubert, rmvpe, MDXNET)
# Эта ячейка создаст каталоги mdxnet_models/ и rvc_models/ и загрузит веса.
import os, subprocess, sys
from pathlib import Path

os.chdir(Path("/content/AiCoverSing2"))
subprocess.run([sys.executable, "src/download_models.py"], check=True)


In [ ]:
# @title 3. (Опционально) Скачать свою RVC-модель по ссылке
# Поддерживаются прямые ссылки и Mega (библиотека mega.py уже установлена).
# Для Mega достаточно вставить полную ссылку вида https://mega.nz/file/...
MODEL_URL = ""  # @param {type:"string"}
MODEL_NAME = "custom_model"  # @param {type:"string"}

from pathlib import Path
from src.download_models import download_from_url

if MODEL_URL:
    target_dir = Path("rvc_models") / MODEL_NAME
    target_dir.mkdir(parents=True, exist_ok=True)
    filename = target_dir / Path(MODEL_URL).name
    print(f"Скачиваем модель в {filename} ...")
    download_from_url(MODEL_URL, str(filename))
    print("Готово! При необходимости распакуйте архив вручную в ту же папку.")
else:
    print("Ссылка не указана — этот шаг можно пропустить, если уже есть модели в rvc_models/.")


In [ ]:
# @title 4. Запуск веб-интерфейса (Gradio)
# После запуска появится публичная ссылка вида https://xxxx.gradio.live
import os, subprocess, sys
from pathlib import Path

os.chdir(Path("/content/AiCoverSing2"))
subprocess.run([sys.executable, "src/webui.py", "--share"], check=True)
